In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/ajiteshshree/Spaceship-Titanic/main/spaceship-titanic/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/ajiteshshree/Spaceship-Titanic/main/spaceship-titanic/test.csv")

In [3]:
test_pid = df_test['PassengerId'].tolist()
def slice_ids(df):
    pid = df['PassengerId'].tolist()
    df['PassengerId'] = [int(x[0:4]) for x in pid]
    df['PersonNo'] = [int(x[5:]) for x in pid]
    return df

In [4]:
df = slice_ids(df)
df_test = slice_ids(df_test)

In [5]:
df.drop('Name',axis=1,inplace=True)
df_test.drop('Name',axis=1,inplace=True)

In [6]:
print("df.shape : ", df.shape)
print("df_test.shape : ", df_test.shape)

df.shape :  (8693, 14)
df_test.shape :  (4277, 13)


In [7]:
#filling NaN values
def preprocess(df):
    li = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck', 'Age']
    for col in li:
        df[col].fillna(df[col].mean(),inplace=True)
    df["HomePlanet"].fillna(df["HomePlanet"].mode()[0],inplace=True)
    df["CryoSleep"].fillna(df["CryoSleep"].mode()[0],inplace=True)
    df["Destination"].fillna(df["Destination"].mode()[0],inplace=True)
    df["VIP"].fillna(df["VIP"].mode()[0],inplace=True)
    df["Cabin"].fillna("",inplace=True)

In [8]:
preprocess(df)
preprocess(df_test)

In [9]:
print("Number of nulls in cabin : ",df['Cabin'].isnull().sum())
df['Cabin'].mode()

Number of nulls in cabin :  0


0    
Name: Cabin, dtype: object

In [10]:
y = df['Transported']
df.drop(['Transported'], axis=1, inplace=True)

In [11]:
print([x for x in list(df.isnull().sum()) if x != 0], end=",")
print([x for x in list(df_test.isnull().sum()) if x != 0])

[],[]


In [12]:
def slice_cabin(df):
    b=0
    cabin_deck =[]
    cabin_side = []
    while b < df.shape[0]:
        if df.iloc[b,3] != "":
            cabin_deck.append(df.iloc[b,3][0])
            cabin_side.append(df.iloc[b,3][-1])
        else:
            cabin_deck.append("")
            cabin_side.append("")
        b += 1
    df['cabin_deck'] = cabin_deck
    df['cabin_side'] = cabin_side
    df.drop('Cabin', axis=1, inplace=True)

In [13]:
slice_cabin(df)
slice_cabin(df_test)

In [14]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PersonNo,cabin_deck,cabin_side
0,13,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,1,G,S
1,18,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,1,F,S
2,19,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,1,C,S
3,21,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,1,C,S
4,23,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,1,F,S


In [15]:
# cols = df.columns
# cols2 = df_test.columns
# for x in cols:
#         print(x,end = ' : ')
#         print(df[x].unique())
# print()
# for t in cols2:
#     print(t,end = ' : ')
#     print(df_test[t].unique())       

In [16]:
non_int_cols = [k for k, v in dict(df.dtypes).items() if v == 'O' or v == 'bool']

In [17]:
non_int_cols

['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck', 'cabin_side']

In [18]:
def encode(df):
#     le = LabelEncoder()    
#     for x in non_int_cols:
#         df[x] = le.fit_transform(df[x])
    df = pd.get_dummies(df, columns=non_int_cols)
    return df

In [19]:
df = encode(df)
df_test = encode(df_test)

In [20]:
df.head()

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PersonNo,HomePlanet_Earth,HomePlanet_Europa,...,cabin_deck_B,cabin_deck_C,cabin_deck_D,cabin_deck_E,cabin_deck_F,cabin_deck_G,cabin_deck_T,cabin_side_,cabin_side_P,cabin_side_S
0,1,39.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
1,2,24.0,109.0,9.0,25.0,549.0,44.0,1,1,0,...,0,0,0,0,1,0,0,0,0,1
2,3,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,3,33.0,0.0,1283.0,371.0,3329.0,193.0,2,0,1,...,0,0,0,0,0,0,0,0,0,1
4,4,16.0,303.0,70.0,151.0,565.0,2.0,1,1,0,...,0,0,0,0,1,0,0,0,0,1


In [21]:
df.columns.tolist()

['PassengerId',
 'Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'PersonNo',
 'HomePlanet_Earth',
 'HomePlanet_Europa',
 'HomePlanet_Mars',
 'CryoSleep_False',
 'CryoSleep_True',
 'Destination_55 Cancri e',
 'Destination_PSO J318.5-22',
 'Destination_TRAPPIST-1e',
 'VIP_False',
 'VIP_True',
 'cabin_deck_',
 'cabin_deck_A',
 'cabin_deck_B',
 'cabin_deck_C',
 'cabin_deck_D',
 'cabin_deck_E',
 'cabin_deck_F',
 'cabin_deck_G',
 'cabin_deck_T',
 'cabin_side_',
 'cabin_side_P',
 'cabin_side_S']

In [22]:
df.shape

(8693, 30)

In [23]:
df_test.shape

(4277, 30)

In [24]:
# df, df_test = df.align(df_test, join='left', axis=1, fill_value=0)

In [25]:
logisticRegr = LogisticRegression(max_iter=10000)
logisticRegr.fit(df,y)
print(logisticRegr.score(df, y))

0.7917864948809387


In [26]:
res = logisticRegr.predict(df_test).tolist()

In [27]:
predictions = pd.DataFrame({'PassengerId' : test_pid,'Transported':res})

In [28]:
predictions.set_index('PassengerId')

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [29]:
predictions.to_csv('predictions.csv',index = False)